# frequency

> Functionality for frequency analysis.

In [ ]:
#| default_exp frequency

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import time
import polars as pl
from fastcore.basics import patch

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE, set_logger_state

In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'

In [ ]:
#| export
class Frequency:
	""" Class for frequency analysis reporting """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| hide
# now working from new vocab data structure
brown = Corpus().load(path_to_brown_corpus)
display(brown.vocab.head(5).collect(engine='streaming'))

rank,token_id,token,frequency_lower,frequency_orth,is_punct,is_space
1,22848,"""the""",63516,62473,false,false
2,8128,""",""",58331,58331,true,false
3,38309,""".""",49907,49907,true,false
4,2739,"""of""",36321,36122,false,false
5,7126,"""and""",27787,27633,false,false


In [ ]:
#| export
@patch
def frequencies(self: Frequency,
				case_sensitive:bool=False, # frequencies for tokens with or without case preserved 
				normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
				page_size:int=PAGE_SIZE, # number of rows to return, if 0 returns all
				page_current:int=1, # current page, ignored if page_size is 0
				show_token_id:bool=False, # show token_id in output
				show_document_frequency:bool=False, # show document frequency in output
				exclude_tokens:list[str]=[], # exclude specific tokens from frequency report, can be used to remove stopwords
				exclude_tokens_text:str = '', # text to explain which tokens have been excluded, will be added to the report notes
				restrict_tokens:list[str]=[], # restrict frequency report to return frequencies for a list of specific tokens
				restrict_tokens_text:str = '', # text to explain which tokens are included, will be added to the report notes
				exclude_punctuation:bool=True, # exclude punctuation tokens
				exclude_spaces:bool=True # exclude space tokens
				) -> Result: # return a Result object with the frequency table
	""" Report frequent tokens. """

	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	start_time = time.time()

	if case_sensitive:
		frequency_column = 'frequency_orth'
		document_count_column = 'orth_index'
	else:
		frequency_column = 'frequency_lower'
		document_count_column = 'lower_index'

	if page_size == 0:
		page_current = 1 # if returning all, then only interested in first page

	columns = ['rank', 'token_id', 'token', 'frequency']

	count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation, exclude_spaces)

	formatted_data = []
	formatted_data.append(f'Report based on {tokens_descriptor}')

	df = self.corpus.vocab.filter(pl.col(frequency_column).is_not_null())
	if exclude_tokens:
		excluded_tokens_count = df.filter(pl.col('token').is_in(exclude_tokens)).select(pl.len()).collect(engine='streaming').item()
		df = df.filter(~pl.col('token').is_in(exclude_tokens))
		if exclude_tokens_text == '':
			formatted_data.append(f'Tokens excluded from report: {excluded_tokens_count}')
		else:
			formatted_data.append(f'{exclude_tokens_text}')
	if restrict_tokens:
		df = df.filter(pl.col('token').is_in(restrict_tokens))
		if restrict_tokens_text == '':
			formatted_data.append(f'')
		else:
			formatted_data.append(f'{restrict_tokens_text}')

	if exclude_punctuation:
		df = df.filter(pl.col('is_punct') == False)
	if exclude_spaces:
		df = df.filter(pl.col('is_space') == False)

	df = df.sort(by = frequency_column, descending=True)

	unique_tokens = df.select(pl.len()).collect(engine='streaming').item()

	if page_size == 0:
		df = df.rename({frequency_column: "frequency"}).select(*columns)
		rank_offset = 1 # not really needed, but just in case future changes
	else:
		df = df.slice((page_current-1)*page_size, page_size).rename({frequency_column: "frequency"}).select(*columns)
		rank_offset = (page_current-1) * page_size+1

	if show_document_frequency:
		document_counts = self.corpus.tokens.select(pl.col(document_count_column).alias('token_id'), pl.col('token2doc_index')).group_by('token_id').agg(pl.col('token2doc_index').n_unique().alias('document_frequency'))
		df = df.join(document_counts, on='token_id', how='left', maintain_order='left')

	df = df.with_columns(((pl.col("frequency") / count_tokens) * normalize_by).alias('normalized_frequency'))
	columns.append('normalized_frequency')

	df = df.drop('rank').with_row_index(name='rank', offset=rank_offset)

	if show_token_id == False:
		df = df.drop('token_id')

	if normalize_by is not None:
		formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	formatted_data.append(f'{total_descriptor}: {count_tokens:,.0f}')

	formatted_data.append(f'Unique tokens: {unique_tokens:,.0f}')
	if page_size != 0 and unique_tokens > page_size:
		formatted_data.append(f'Showing {page_size} rows')
		formatted_data.append(f'Page {page_current} of {unique_tokens // page_size + 1}')

	logger.info(f'Frequencies report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'frequencies', df=df, title='Frequencies', description=f'Frequencies of {tokens_descriptor}, {self.corpus.name}', summary_data={}, formatted_data=formatted_data)


In [ ]:
# load the corpus
brown = Corpus().load(path_to_brown_corpus)

In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)

In [ ]:
#| hide
toy.summary()
brown.summary()

Corpus Summary 
 
 
 
 
 
 Attribute 
 Value 
 
 
 
 
 Name 
 Toy Corpus 
 
 
 Description 
 Toy corpus is a very small dataset for testing and library development. 
 
 
 Date Created 
 2025-06-08 20:22:21 
 
 
 Conc Version 
 0.0.1 
 
 
 Corpus Path 
 /home/geoff/data/conc-test-corpora/toy.corpus 
 
 
 Document Count 
 6 
 
 
 Token Count 
 38 
 
 
 Word Token Count 
 32 
 
 
 Unique Tokens 
 15 
 
 
 Unique Word Tokens 
 14

Corpus Summary 
 
 
 
 
 
 Attribute 
 Value 
 
 
 
 
 Name 
 Brown Corpus 
 
 
 Description 
 A Standard Corpus of Present-Day Edited American English, for use with Digital Computers. by W. N. Francis and H. Kucera (1964) Department of Linguistics, Brown University Providence, Rhode Island, USA Revised 1971, Revised and Amplified 1979 http://www.hit.uib.no/icame/brown/bcm.html. This version downloaded via NLTK https://www.nltk.org/nltk_data/. 
 
 
 Date Created 
 2025-06-08 20:22:21 
 
 
 Conc Version 
 0.0.1 
 
 
 Corpus Path 
 /home/geoff/data/conc-test-corpora/brown.corpus 
 
 
 Document Count 
 500 
 
 
 Token Count 
 1,140,905 
 
 
 Word Token Count 
 980,144 
 
 
 Unique Tokens 
 42,937 
 
 
 Unique Word Tokens 
 42,907

In [ ]:
# instantiate the Frequency class
freq_brown = Frequency(brown)

In [ ]:
#| hide
from conc.core import show_toy_corpus

In [ ]:
#| hide
show_toy_corpus(f'{source_path}toy.csv')

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| hide
freq_toy = Frequency(toy)
frequencies_test = freq_toy.frequencies(normalize_by=100, show_document_frequency = True, page_size=20).to_frame().to_dict(as_series=False)
normalized_frequencies_test = dict(zip(frequencies_test['token'], frequencies_test['normalized_frequency']))
document_frequencies_test = dict(zip(frequencies_test['token'], frequencies_test['document_frequency']))
frequencies_test = dict(zip(frequencies_test['token'], frequencies_test['frequency']))
assert frequencies_test['is'] == 4
assert normalized_frequencies_test['is'] == ((4/32) * 100)
assert document_frequencies_test['is'] == 4

%time freq_toy.frequencies(normalize_by=100, show_document_frequency = True, page_size=20).display()



Frequencies 
 
 
 Frequencies of word tokens, Toy Corpus 
 
 
 Rank 
 Token 
 Frequency 
 Document Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 8 
 6 
 25.00 
 
 
 2 
 is 
 4 
 4 
 12.50 
 
 
 3 
 cat 
 3 
 3 
 9.38 
 
 
 4 
 dog 
 3 
 3 
 9.38 
 
 
 5 
 on 
 2 
 2 
 6.25 
 
 
 6 
 sat 
 2 
 2 
 6.25 
 
 
 7 
 mat 
 2 
 2 
 6.25 
 
 
 8 
 a 
 2 
 2 
 6.25 
 
 
 9 
 climbing 
 1 
 1 
 3.12 
 
 
 10 
 barking 
 1 
 1 
 3.12 
 
 
 11 
 tree 
 1 
 1 
 3.12 
 
 
 12 
 meowing 
 1 
 1 
 3.12 
 
 
 13 
 digging 
 1 
 1 
 3.12 
 
 
 14 
 hole 
 1 
 1 
 3.12 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 100 tokens 
 


 
 Total word tokens: 32 
 


 
 Unique tokens: 14

CPU times: user 32 ms, sys: 11.7 ms, total: 43.7 ms
Wall time: 15 ms


In [ ]:
# run the frequencies method and display the results
freq_brown.frequencies(normalize_by=10000, page_size=20).display()

Frequencies 
 
 
 Frequencies of word tokens, Brown Corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 63,516 
 648.03 
 
 
 2 
 of 
 36,321 
 370.57 
 
 
 3 
 and 
 27,787 
 283.50 
 
 
 4 
 to 
 25,868 
 263.92 
 
 
 5 
 a 
 22,190 
 226.40 
 
 
 6 
 in 
 19,751 
 201.51 
 
 
 7 
 that 
 10,409 
 106.20 
 
 
 8 
 is 
 10,138 
 103.43 
 
 
 9 
 was 
 9,931 
 101.32 
 
 
 10 
 for 
 8,905 
 90.85 
 
 
 11 
 with 
 7,043 
 71.86 
 
 
 12 
 it 
 6,991 
 71.33 
 
 
 13 
 he 
 6,772 
 69.09 
 
 
 14 
 as 
 6,738 
 68.75 
 
 
 15 
 his 
 6,523 
 66.55 
 
 
 16 
 on 
 6,459 
 65.90 
 
 
 17 
 be 
 6,365 
 64.94 
 
 
 18 
 's 
 5,285 
 53.92 
 
 
 19 
 had 
 5,200 
 53.05 
 
 
 20 
 by 
 5,156 
 52.60 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens: 980,144 
 


 
 Unique tokens: 42,907 
 


 
 Showing 20 rows 
 


 
 Page 1 of 2146

In [ ]:
#| hide
# test page 2
freq_brown.frequencies(normalize_by=10000, page_current = 2, page_size=20).display()

Frequencies 
 
 
 Frequencies of word tokens, Brown Corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 21 
 at 
 4,990 
 50.91 
 
 
 22 
 not 
 4,726 
 48.22 
 
 
 23 
 are 
 4,391 
 44.80 
 
 
 24 
 i 
 4,370 
 44.59 
 
 
 25 
 from 
 4,228 
 43.14 
 
 
 26 
 or 
 4,100 
 41.83 
 
 
 27 
 this 
 4,037 
 41.19 
 
 
 28 
 have 
 3,950 
 40.30 
 
 
 29 
 an 
 3,570 
 36.42 
 
 
 30 
 which 
 3,546 
 36.18 
 
 
 31 
 were 
 3,304 
 33.71 
 
 
 32 
 you 
 3,265 
 33.31 
 
 
 33 
 one 
 3,107 
 31.70 
 
 
 34 
 but 
 2,969 
 30.29 
 
 
 35 
 her 
 2,887 
 29.45 
 
 
 36 
 all 
 2,875 
 29.33 
 
 
 37 
 they 
 2,842 
 29.00 
 
 
 38 
 would 
 2,816 
 28.73 
 
 
 39 
 him 
 2,619 
 26.72 
 
 
 40 
 their 
 2,556 
 26.08 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens: 980,144 
 


 
 Unique tokens: 42,907 
 


 
 Showing 20 rows 
 


 
 Page 2 of 2146

In [ ]:
from conc.core import get_stop_words
stop_words = get_stop_words(save_path, spacy_model = 'en_core_web_sm')
freq_brown.frequencies(normalize_by=10000, show_document_frequency = True, exclude_tokens = stop_words, page_size=20).display()

Frequencies 
 
 
 Frequencies of word tokens, Brown Corpus 
 
 
 Rank 
 Token 
 Frequency 
 Document Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 said 
 1,944 
 315 
 19.83 
 
 
 2 
 time 
 1,667 
 450 
 17.01 
 
 
 3 
 new 
 1,595 
 390 
 16.27 
 
 
 4 
 man 
 1,346 
 326 
 13.73 
 
 
 5 
 like 
 1,287 
 366 
 13.13 
 
 
 6 
 af 
 989 
 49 
 10.09 
 
 
 7 
 years 
 953 
 346 
 9.72 
 
 
 8 
 way 
 925 
 365 
 9.44 
 
 
 9 
 state 
 883 
 200 
 9.01 
 
 
 10 
 long 
 863 
 354 
 8.80 
 
 
 11 
 people 
 851 
 286 
 8.68 
 
 
 12 
 world 
 848 
 274 
 8.65 
 
 
 13 
 year 
 831 
 242 
 8.48 
 
 
 14 
 little 
 823 
 322 
 8.40 
 
 
 15 
 good 
 813 
 320 
 8.29 
 
 
 16 
 men 
 772 
 248 
 7.88 
 
 
 17 
 work 
 767 
 310 
 7.83 
 
 
 18 
 day 
 767 
 311 
 7.83 
 
 
 19 
 old 
 734 
 278 
 7.49 
 
 
 20 
 life 
 728 
 284 
 7.43 
 
 
 
 
 
 Report based on word tokens 
 


 
 Tokens excluded from report: 306 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens: 980,144 
 


 
 Unique tokens: 42,601 
 


 
 Showing 20 rows 
 


 
 Page 1 of 2131

In [ ]:
#| hide
congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-100k.corpus')
freq_congress = Frequency(congress)
%time freq_congress.frequencies(normalize_by=10000, page_size=20, show_document_frequency=True, exclude_tokens = stop_words).display()

Frequencies 
 
 
 Frequencies of word tokens, US Congressional Speeches Subset 100k 
 
 
 Rank 
 Token 
 Frequency 
 Document Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 mr. 
 89,682 
 54,158 
 49.77 
 
 
 2 
 bill 
 69,300 
 31,379 
 38.46 
 
 
 3 
 senator 
 52,772 
 25,066 
 29.28 
 
 
 4 
 committee 
 48,387 
 25,583 
 26.85 
 
 
 5 
 president 
 47,323 
 28,791 
 26.26 
 
 
 6 
 time 
 40,065 
 26,874 
 22.23 
 
 
 7 
 amendment 
 39,940 
 18,537 
 22.16 
 
 
 8 
 states 
 34,700 
 19,839 
 19.26 
 
 
 9 
 house 
 33,129 
 19,134 
 18.38 
 
 
 10 
 gentleman 
 32,178 
 19,140 
 17.86 
 
 
 11 
 speaker 
 31,284 
 21,512 
 17.36 
 
 
 12 
 senate 
 30,837 
 17,535 
 17.11 
 
 
 13 
 people 
 30,708 
 17,273 
 17.04 
 
 
 14 
 state 
 27,747 
 16,052 
 15.40 
 
 
 15 
 think 
 27,673 
 18,935 
 15.36 
 
 
 16 
 congress 
 26,960 
 16,452 
 14.96 
 
 
 17 
 new 
 26,810 
 16,197 
 14.88 
 
 
 18 
 united 
 26,228 
 15,802 
 14.55 
 
 
 19 
 government 
 25,685 
 14,805 
 14.25 
 
 
 20 
 years 
 25,233 
 16,497 
 14.00 
 
 
 
 
 
 Report based on word tokens 
 


 
 Tokens excluded from report: 303 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens: 18,020,769 
 


 
 Unique tokens: 213,872 
 


 
 Showing 20 rows 
 


 
 Page 1 of 10694

CPU times: user 2.72 s, sys: 495 ms, total: 3.22 s
Wall time: 291 ms


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()